# Auto Scout Project 
**Magnimind** 

**Author:** Mark    
**Date:** February 2026  
**GitHub:** [link](https://github.com/mjoslin-ai/Magnimind-Project-Auto-Scout)

In [713]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Project Phase:
### 2. Missing Value Analysis

#### Data Exploration

In [ ]:
df = pd.read_csv("phase1_cleaned_data.csv")
print(df.shape)
print(df.head())
print(df.info())

(15919, 54)
                                                                                                                                                url  \
0         https://www.autoscout24.com//offers/audi-a1-sportback-1-4-tdi-s-tronic-xenon-navi-klima-diesel-black-bdab349a-caa5-41b0-98eb-c1345b84445e   
1                                      https://www.autoscout24.com//offers/audi-a1-1-8-tfsi-sport-gasoline-red-b2547f8a-e83f-6237-e053-e250040a56df   
2  https://www.autoscout24.com//offers/audi-a1-sportback-1-6-tdi-s-tronic-einparkhilfe-plus-music-diesel-black-6183cb6a-8570-4b86-a132-9b54214bca88   
3                             https://www.autoscout24.com//offers/audi-a1-1-4-tdi-design-s-tronic-diesel-brown-35c86585-2727-d61e-e053-e250040ad640   
4    https://www.autoscout24.com//offers/audi-a1-sportback-1-4-tdi-s-tronic-s-line-ext-admired-xe-diesel-black-b86b55d8-9c7c-4a58-b658-b97008a51efc   

  make_model                                   short_description body_type  \
0  

#### Helper Functions

#### Clean Quantitative Features

#### Clean Categorical Features

#### Final Dataset